# Exploring 22 Records Referencing Runways (Kickback from 302 records sent to Lex)
***
***DAEN690***

***George Mason University***

***Author:*** Grace Cox (Team LEGO)

***Date:*** November 15, 2021
***
`This Jupyter Notebook explores 22 records (of 302 records) that reference runways within the remarks. The process for exploring these records is as follows:`

1. Drop records with field entries that are null
2. Merge remaining records with the airports_runways_linked.csv file, which contains the lat/long coordinates of runways at select airports
3. Extract Runway Designator from the Runway Location
4. Check if the Runway Designator found within the remark links/matches with one of the runway in the airports_runways_linked.csv dataset
    a. Those records for which there is no link to the aforementioned supplemental dataset will be moved to the exceptions file, as there is no runway with that specific name at the airport mentioned.
5. Link with the airports_cleaned_declination.csv in order to obtain the declination
6. Calculate bearing information for runways
7. Convert Distance from NM to Kilometers
    a. If NO DISTANCE information is provided in the remark, the Runway lat/long coordinates are used for the UAS lat/long coordinates
8. Calculate UAS latitude/longitude coordinated using geopy

***
### INPUT: 
*Dataframe (read from a .csv file) containing 22 records, of 302 records that did not hit on the initial 'XX NM dir XXX' regular expressions, but do reference a runway.*

### OUTPUT:
<font color='red'>**EXCPETIONS FILE**</font>

*Runways (no distance information or no match to airports/runways dataset): 2 records*

<font color='green'>**UAS LATITUDE/LONGITUDE COORDINATE CALCULATED**</font>

*Runway: 20 records*

## Import Statements

In [25]:
# Import Statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import chart_studio.plotly as py
import re
import geopy
from geopy.distance import geodesic
from fractions import Fraction

from IPython.display import display, HTML

In [4]:
rwy22_allrows = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/Grace_All_Points_rwy302_allrows.csv')
rwy22_allrows

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance,dateonly,timeonly,DATE,...,UASFORMATION.CEDAR,CLOSESTPROXIMITY.CEDAR,UASWEIGHTGT55.CEDAR,UASDIM.CEDAR,UASFWROTOR.CEDAR,PILOTREPORTEDNMAC.CEDAR,TCASRA.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,DATASET
0,433,Aircraft reported a uasmaneuvering 300 feet be...,['RWY7R'],1 NM S DVT,NaN,RWY 07R,['1 NM'],1/8/2018,2:42,2018-01-08T02:42Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY
1,452,Aircraft reported a black and red four rotor u...,['RWY9L'],1 NM W of OPF,NaN,RWY 09L,['1NM'],3/1/2018,22:20,2018-03-01T22:20Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY
2,473,Aircraft reported a shiny uas at same altitude...,['RWY18'],1/2 NM S PIE,NaN,RWY 18,['2 NM'],4/3/2018,22:15,2018-04-03T22:15Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY
3,507,Aircraft reported a white 4 rotor uas approxim...,['RWY31L'],6 NM ESE of RHV,NaN,RWY 31L,['6 NM'],6/14/2018,10:18,2018-06-14T10:18Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY
4,534,Aircraft reported a uas off the left side at 1...,['RWY22'],6 NM NE LGA,NaN,RWY 22,[],10/17/2018,0:47,2018-10-17T00:47Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY
5,544,Aircraft reported a uas hovering at approximat...,['RWY10R'],1 NM S of FLL,NaN,RWY 10R,['2NM'],12/25/2018,10:28,2018-12-25T10:28Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY
6,565,Aircraft reported a white and brown quad-copte...,"['RWY 5R', 'RWY']",2 NM SW of RDU,NaN,RWY 05R,"['2NM', '2NM']",5/14/2019,23:18,2019-05-14T23:18Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY
7,126,Aircraft observed a black uas while SW bound a...,['RWY 24R'],2 NM E of LAX,NaN,RWY 24R,['2 NM'],11/29/2019,16:43,2019-11-29T16:43Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY
8,205,Aircraft observed a uas while NE bound in a le...,['RWY 22'],1 NM E of EAU,NaN,RWY 22,['1NM'],7/4/2020,15:36,2020-07-04T15:36Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY
9,252,Aircraft observed a uas below while W bound on...,['RWY 7'],2 NM N of ORL,NaN,RWY 07,['2 NM'],9/27/2020,12:48,2020-09-27T12:48Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY


In [10]:
rwy22 = rwy22_allrows[['REMARKS', 'UASLOCATION', 'IDENT', 'RWYLOCATION']]
rwy22

,REMARKS,UASLOCATION,IDENT,RWYLOCATION
0,Aircraft reported a uasmaneuvering 300 feet be...,1 NM S DVT,NaN,RWY 07R
1,Aircraft reported a black and red four rotor u...,1 NM W of OPF,NaN,RWY 09L
2,Aircraft reported a shiny uas at same altitude...,1/2 NM S PIE,NaN,RWY 18
3,Aircraft reported a white 4 rotor uas approxim...,6 NM ESE of RHV,NaN,RWY 31L
4,Aircraft reported a uas off the left side at 1...,6 NM NE LGA,NaN,RWY 22
5,Aircraft reported a uas hovering at approximat...,1 NM S of FLL,NaN,RWY 10R
6,Aircraft reported a white and brown quad-copte...,2 NM SW of RDU,NaN,RWY 05R
7,Aircraft observed a black uas while SW bound a...,2 NM E of LAX,NaN,RWY 24R
8,Aircraft observed a uas while NE bound in a le...,1 NM E of EAU,NaN,RWY 22
9,Aircraft observed a uas below while W bound on...,2 NM N of ORL,NaN,RWY 07


In [11]:
for i in range(len(rwy22)):
    rwy22['IDENT'].loc[i] = rwy22['UASLOCATION'].loc[i].split()[-1]
rwy22

C:\Users\grace\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-11-b905e2e242af>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rwy22['IDENT'].loc[i] = rwy22['UASLOCATION'].loc[i].split()[-1]


,REMARKS,UASLOCATION,IDENT,RWYLOCATION
0,Aircraft reported a uasmaneuvering 300 feet be...,1 NM S DVT,DVT,RWY 07R
1,Aircraft reported a black and red four rotor u...,1 NM W of OPF,OPF,RWY 09L
2,Aircraft reported a shiny uas at same altitude...,1/2 NM S PIE,PIE,RWY 18
3,Aircraft reported a white 4 rotor uas approxim...,6 NM ESE of RHV,RHV,RWY 31L
4,Aircraft reported a uas off the left side at 1...,6 NM NE LGA,LGA,RWY 22
5,Aircraft reported a uas hovering at approximat...,1 NM S of FLL,FLL,RWY 10R
6,Aircraft reported a white and brown quad-copte...,2 NM SW of RDU,RDU,RWY 05R
7,Aircraft observed a black uas while SW bound a...,2 NM E of LAX,LAX,RWY 24R
8,Aircraft observed a uas while NE bound in a le...,1 NM E of EAU,EAU,RWY 22
9,Aircraft observed a uas below while W bound on...,2 NM N of ORL,ORL,RWY 07


In [12]:
rwy22

,REMARKS,UASLOCATION,IDENT,RWYLOCATION
0,Aircraft reported a uasmaneuvering 300 feet be...,1 NM S DVT,DVT,RWY 07R
1,Aircraft reported a black and red four rotor u...,1 NM W of OPF,OPF,RWY 09L
2,Aircraft reported a shiny uas at same altitude...,1/2 NM S PIE,PIE,RWY 18
3,Aircraft reported a white 4 rotor uas approxim...,6 NM ESE of RHV,RHV,RWY 31L
4,Aircraft reported a uas off the left side at 1...,6 NM NE LGA,LGA,RWY 22
5,Aircraft reported a uas hovering at approximat...,1 NM S of FLL,FLL,RWY 10R
6,Aircraft reported a white and brown quad-copte...,2 NM SW of RDU,RDU,RWY 05R
7,Aircraft observed a black uas while SW bound a...,2 NM E of LAX,LAX,RWY 24R
8,Aircraft observed a uas while NE bound in a le...,1 NM E of EAU,EAU,RWY 22
9,Aircraft observed a uas below while W bound on...,2 NM N of ORL,ORL,RWY 07


In [30]:
dist_list = []

for i in range(len(rwy22)):
    uas_dist = rwy22['UASLOCATION'].loc[i].split()[0]
    dist_list.append(uas_dist)
    
for i in range(len(dist_list)):
    if '/' in dist_list[i]:
        x = Fraction(dist_list[i])
        dist_list[i] = float(x)

rwy22['Distance_NM'] = dist_list

<ipython-input-30-9133590d5ed8>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rwy22['Distance_NM'] = dist_list


In [31]:
rwy22

,REMARKS,UASLOCATION,IDENT,RWYLOCATION,Distance_NM
0,Aircraft reported a uasmaneuvering 300 feet be...,1 NM S DVT,DVT,RWY 07R,1
1,Aircraft reported a black and red four rotor u...,1 NM W of OPF,OPF,RWY 09L,1
2,Aircraft reported a shiny uas at same altitude...,1/2 NM S PIE,PIE,RWY 18,0.5
3,Aircraft reported a white 4 rotor uas approxim...,6 NM ESE of RHV,RHV,RWY 31L,6
4,Aircraft reported a uas off the left side at 1...,6 NM NE LGA,LGA,RWY 22,6
5,Aircraft reported a uas hovering at approximat...,1 NM S of FLL,FLL,RWY 10R,1
6,Aircraft reported a white and brown quad-copte...,2 NM SW of RDU,RDU,RWY 05R,2
7,Aircraft observed a black uas while SW bound a...,2 NM E of LAX,LAX,RWY 24R,2
8,Aircraft observed a uas while NE bound in a le...,1 NM E of EAU,EAU,RWY 22,1
9,Aircraft observed a uas below while W bound on...,2 NM N of ORL,ORL,RWY 07,2


In [50]:
def drop_nan(dataframe):
    '''
    This function replaces all instances of [] in the dataframe provided as
    input with NaN and then returns a new dataframe with no 'NaN' values
    
    input : dataframe is a dataframe containing UAS remark information
    output : a new dataframe with no 'NaN' values
    '''
    
    df1 = dataframe.mask(dataframe.applymap(str).eq('[]'))
    output_df = df1.dropna().reset_index(drop=True)
    
    return output_df

In [52]:
rwy21 = drop_nan(rwy22)
rwy21

,REMARKS,UASLOCATION,IDENT,RWYLOCATION,Distance_NM
0,Aircraft reported a uasmaneuvering 300 feet be...,1 NM S DVT,DVT,RWY 07R,1
1,Aircraft reported a black and red four rotor u...,1 NM W of OPF,OPF,RWY 09L,1
2,Aircraft reported a shiny uas at same altitude...,1/2 NM S PIE,PIE,RWY 18,0.5
3,Aircraft reported a white 4 rotor uas approxim...,6 NM ESE of RHV,RHV,RWY 31L,6
4,Aircraft reported a uas off the left side at 1...,6 NM NE LGA,LGA,RWY 22,6
5,Aircraft reported a uas hovering at approximat...,1 NM S of FLL,FLL,RWY 10R,1
6,Aircraft reported a white and brown quad-copte...,2 NM SW of RDU,RDU,RWY 05R,2
7,Aircraft observed a black uas while SW bound a...,2 NM E of LAX,LAX,RWY 24R,2
8,Aircraft observed a uas while NE bound in a le...,1 NM E of EAU,EAU,RWY 22,1
9,Aircraft observed a uas below while W bound on...,2 NM N of ORL,ORL,RWY 07,2


In [55]:
def airRwy_link (dataframe):
    '''
    This function does the following:
        1. Merge the dataframe with the airports_runways_linked.csv dataset
            a. Merge in the IDENT field
        2. Extracts the Runway Designator from RWYLOCATION field
        3. Checks if the Runway Designator found in the Remark matches one
           of the RWY Designators from airports_runways_linked.csv
            a. Only keeps those records for which there is a match
        4. Merge the resulting dataframe with airports_cleaned_declination.csv
           to get the declination for the runway
    
    input : dataframe is a dataframe containing UAS remark information and 
            Runway Location information in a field named RWYLOCATION
    output : a new dataframe in which all records have a runway location
             that is present at an airport in airports_runways_linked.csv
    '''
    # Import and store datasets to merge with
    air_rwy = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airports_runways_linked.csv')
    air_dec = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airports_cleaned_declination.csv')
    
    # Merge the two datasets on the 'IDENT' field
    rwy_linked = pd.merge(dataframe, air_rwy, on = 'IDENT', how = 'left')
    rwy_linked.rename(columns = {'properties.DESIGNATOR': 'DESIGNATOR'}, inplace = True)
    
    # Extract Runway Designator from RWYLOCATION field
    rwydesignator = []

    for i in range(len(rwy_linked)):
        rwy_split = rwy_linked['RWYLOCATION'][i].split(' ')
        rwydesignator.append(rwy_split[1])

    # Add split column with RWY designators to dataframe as new field
    rwy_linked['RWYDESIGNATOR'] = rwydesignator
    
    # Check if Runway Designator found in Remark matches one of the Runway designators
    # from the air_rwy dataframe
    rwy_linked['RWYinREMARK'] = rwy_linked.apply(lambda x: str(x.RWYDESIGNATOR) in str(x.DESIGNATOR), axis=1)

    # Only keep records for which the RWY Designator is present in the DESIGNATOR field 
    output_df1 = rwy_linked[rwy_linked['RWYinREMARK'] == True]
    
    # Merge resulting dataframe with airport declination (air_dec) dataframe
    output_dec = pd.merge(output_df1, air_dec, on = 'IDENT', how = 'left')
    
    # Only keep necessary columns/fields of interest
    output_df = pd.DataFrame(output_dec[['REMARKS', 'UASLOCATION', 'IDENT', 'RWYLOCATION', 'Distance_NM', 'RWYDESIGNATOR', 'Runway.Latitude', 'Runway.Longitude', 'DECLINATION']])
    
    return output_df

In [63]:
rwy20 = airRwy_link(rwy21).drop_duplicates(subset=['REMARKS']).reset_index(drop=True)
rwy20

,REMARKS,UASLOCATION,IDENT,RWYLOCATION,Distance_NM,RWYDESIGNATOR,Runway.Latitude,Runway.Longitude,DECLINATION
0,Aircraft reported a uasmaneuvering 300 feet be...,1 NM S DVT,DVT,RWY 07R,1,07R,-112.085527,33.687439,10.02699
1,Aircraft reported a black and red four rotor u...,1 NM W of OPF,OPF,RWY 09L,1,09L,-80.274087,25.913315,-6.94323
2,Aircraft reported a shiny uas at same altitude...,1/2 NM S PIE,PIE,RWY 18,0.5,18,-82.688023,27.907540,-5.72372
3,Aircraft reported a white 4 rotor uas approxim...,6 NM ESE of RHV,RHV,RWY 31L,6,31L,-121.820886,37.333293,13.05261
4,Aircraft reported a uas off the left side at 1...,6 NM NE LGA,LGA,RWY 22,6,22,-73.878696,40.775660,-12.75395
5,Aircraft reported a uas hovering at approximat...,1 NM S of FLL,FLL,RWY 10R,1,10R,-80.143732,26.065846,-7.04262
6,Aircraft reported a white and brown quad-copte...,2 NM SW of RDU,RDU,RWY 05R,2,05R,-78.790127,35.870421,-9.22834
7,Aircraft observed a black uas while SW bound a...,2 NM E of LAX,LAX,RWY 24R,2,24R,-118.419482,33.950273,11.71212
8,Aircraft observed a uas while NE bound in a le...,1 NM E of EAU,EAU,RWY 22,1,22,-91.483850,44.865552,-1.36005
9,Aircraft observed a uas below while W bound on...,2 NM N of ORL,ORL,RWY 07,2,07,-81.328953,28.546714,-6.64984


In [65]:
def rwyUAS_latLong (dataframe):
    '''
    This function takes in a dataframe containing Runway latitude/longitude
    information as well as runway declination information for each Remark
    having a runway origin reference location.
    
    This function does the following:
        1. Calculates bearing information for each runway reference
        2. Converts distances from NM to kilometers
        3. Calculates UAS Latitude/Longitude information using geopy
    
    input : dataframe containing runway lat/long and declination information
    output : dataframe containing UAS Latitude/Longitude information
    '''
    # Calculate bearing information for each runway reference
    designator = dataframe['RWYDESIGNATOR']
    designator_bearing = []

    for i in range(len(dataframe)):
        designator_bearing.append(int(re.sub("\D","",designator[i]))*10 + dataframe['DECLINATION'][i])

    dataframe['RWY_BEARING'] = designator_bearing
    
    # Convert Distances from NM to kilometers
    dist_kilo = []

    for i in range(len(dataframe)):
        distanceKilo = int(re.sub("\D","",str(dataframe['Distance_NM'][i])))* 1.852 # converting NM to kilometers
        dist_kilo.append(distanceKilo)

    dataframe['Distance_Kilometers'] = dist_kilo
    
    # BECAUSE ERIC HAD THESE SWITCHED ****************
    dataframe.rename(columns = {'Runway.Latitude': 'Runway_Longitude', 'Runway.Longitude' : 'Runway_Latitude'}, inplace = True)
    
    # Calculate UAS Lat/Long information using geopy
    uas_lat = []
    uas_long = []


    for i in range(len(dataframe)):
        lat_rwy = pd.to_numeric(dataframe['Runway_Latitude'][i])
        long_rwy = pd.to_numeric(dataframe['Runway_Longitude'][i])
        b = pd.to_numeric(dataframe['RWY_BEARING'][i])
        d = pd.to_numeric(dataframe['Distance_Kilometers'][i])

        origin = geopy.Point(lat_rwy, long_rwy)
        destination = geodesic(kilometers=d).destination(origin,b)

        lat2, lon2, = destination.latitude, destination.longitude

        uas_lat.append(lat2)
        uas_long.append(lon2)

    # Append UAS Lat/Long information to DataFrame
    dataframe['UAS_Latitude'] = uas_lat
    dataframe['UAS_Longitude'] = uas_long
    
    # Export dataframe containing UAS lat/long information to .csv file 
    dataframe.to_csv('rwy%d_uasLatLong.csv'%len(dataframe), index=False)
    
    return dataframe

In [68]:
rwy20_uas = rwyUAS_latLong(rwy20)
rwy20_uas

,REMARKS,UASLOCATION,IDENT,RWYLOCATION,Distance_NM,RWYDESIGNATOR,Runway_Longitude,Runway_Latitude,DECLINATION,RWY_BEARING,Distance_Kilometers,UAS_Latitude,UAS_Longitude
0,Aircraft reported a uasmaneuvering 300 feet be...,1 NM S DVT,DVT,RWY 07R,1,07R,-112.085527,33.687439,10.02699,80.02699,1.852,33.690329,-112.065855
1,Aircraft reported a black and red four rotor u...,1 NM W of OPF,OPF,RWY 09L,1,09L,-80.274087,25.913315,-6.94323,83.05677,1.852,25.915335,-80.255737
2,Aircraft reported a shiny uas at same altitude...,1/2 NM S PIE,PIE,RWY 18,0.5,18,-82.688023,27.907540,-5.72372,174.27628,9.260,27.824396,-82.678650
3,Aircraft reported a white 4 rotor uas approxim...,6 NM ESE of RHV,RHV,RWY 31L,6,31L,-121.820886,37.333293,13.05261,323.05261,11.112,37.413286,-121.896334
4,Aircraft reported a uas off the left side at 1...,6 NM NE LGA,LGA,RWY 22,6,22,-73.878696,40.775660,-12.75395,207.24605,11.112,40.686682,-73.938877
5,Aircraft reported a uas hovering at approximat...,1 NM S of FLL,FLL,RWY 10R,1,10R,-80.143732,26.065846,-7.04262,92.95738,1.852,26.064982,-80.125248
6,Aircraft reported a white and brown quad-copte...,2 NM SW of RDU,RDU,RWY 05R,2,05R,-78.790127,35.870421,-9.22834,40.77166,3.704,35.895699,-78.763335
7,Aircraft observed a black uas while SW bound a...,2 NM E of LAX,LAX,RWY 24R,2,24R,-118.419482,33.950273,11.71212,251.71212,3.704,33.939789,-118.457523
8,Aircraft observed a uas while NE bound in a le...,1 NM E of EAU,EAU,RWY 22,1,22,-91.483850,44.865552,-1.36005,218.63995,1.852,44.852534,-91.498479
9,Aircraft observed a uas below while W bound on...,2 NM N of ORL,ORL,RWY 07,2,07,-81.328953,28.546714,-6.64984,63.35016,3.704,28.561701,-81.295120


In [70]:
rwy_merge = df_all = rwy22.merge(rwy20_uas.drop_duplicates(), on=['REMARKS'], 
                   how='left', indicator=True)
rwy_merge

,REMARKS,UASLOCATION_x,IDENT_x,RWYLOCATION_x,Distance_NM_x,UASLOCATION_y,IDENT_y,RWYLOCATION_y,Distance_NM_y,RWYDESIGNATOR,Runway_Longitude,Runway_Latitude,DECLINATION,RWY_BEARING,Distance_Kilometers,UAS_Latitude,UAS_Longitude,_merge
0,Aircraft reported a uasmaneuvering 300 feet be...,1 NM S DVT,DVT,RWY 07R,1,1 NM S DVT,DVT,RWY 07R,1,07R,-112.085527,33.687439,10.02699,80.02699,1.852,33.690329,-112.065855,both
1,Aircraft reported a black and red four rotor u...,1 NM W of OPF,OPF,RWY 09L,1,1 NM W of OPF,OPF,RWY 09L,1,09L,-80.274087,25.913315,-6.94323,83.05677,1.852,25.915335,-80.255737,both
2,Aircraft reported a shiny uas at same altitude...,1/2 NM S PIE,PIE,RWY 18,0.5,1/2 NM S PIE,PIE,RWY 18,0.5,18,-82.688023,27.907540,-5.72372,174.27628,9.260,27.824396,-82.678650,both
3,Aircraft reported a white 4 rotor uas approxim...,6 NM ESE of RHV,RHV,RWY 31L,6,6 NM ESE of RHV,RHV,RWY 31L,6,31L,-121.820886,37.333293,13.05261,323.05261,11.112,37.413286,-121.896334,both
4,Aircraft reported a uas off the left side at 1...,6 NM NE LGA,LGA,RWY 22,6,6 NM NE LGA,LGA,RWY 22,6,22,-73.878696,40.775660,-12.75395,207.24605,11.112,40.686682,-73.938877,both
5,Aircraft reported a uas hovering at approximat...,1 NM S of FLL,FLL,RWY 10R,1,1 NM S of FLL,FLL,RWY 10R,1,10R,-80.143732,26.065846,-7.04262,92.95738,1.852,26.064982,-80.125248,both
6,Aircraft reported a white and brown quad-copte...,2 NM SW of RDU,RDU,RWY 05R,2,2 NM SW of RDU,RDU,RWY 05R,2,05R,-78.790127,35.870421,-9.22834,40.77166,3.704,35.895699,-78.763335,both
7,Aircraft observed a black uas while SW bound a...,2 NM E of LAX,LAX,RWY 24R,2,2 NM E of LAX,LAX,RWY 24R,2,24R,-118.419482,33.950273,11.71212,251.71212,3.704,33.939789,-118.457523,both
8,Aircraft observed a uas while NE bound in a le...,1 NM E of EAU,EAU,RWY 22,1,1 NM E of EAU,EAU,RWY 22,1,22,-91.483850,44.865552,-1.36005,218.63995,1.852,44.852534,-91.498479,both
9,Aircraft observed a uas below while W bound on...,2 NM N of ORL,ORL,RWY 07,2,2 NM N of ORL,ORL,RWY 07,2,07,-81.328953,28.546714,-6.64984,63.35016,3.704,28.561701,-81.295120,both


In [76]:
rwy_unid2 = rwy_merge[rwy_merge['_merge'] == 'left_only'].reset_index(drop=True)
rwy_unid2

,REMARKS,UASLOCATION_x,IDENT_x,RWYLOCATION_x,Distance_NM_x,UASLOCATION_y,IDENT_y,RWYLOCATION_y,Distance_NM_y,RWYDESIGNATOR,Runway_Longitude,Runway_Latitude,DECLINATION,RWY_BEARING,Distance_Kilometers,UAS_Latitude,UAS_Longitude,_merge
0,Aircraft observed a quad copter uas off the ri...,.25 NM N of APA,APA,NaN,.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,Aircraft observed a white uas while S bound at...,1 NM N of M01,M01,RWY 18,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [77]:
# Export to .csv file 
rwy_unid2.to_csv('rwy2of302_EXCEPTIONS.csv', index=False)